In [ ]:
# All your existing imports
import jax.numpy as jnp
import jax.random as jr
import matplotlib.pyplot as plt
import time
from config import create_range_parameter_config
from dataset import your_prior_sampler, your_simulator
from nn_model import NCMLP
from sde import get_sde
from train import train_score_network
from cnf import CNF

# MAIN CONFIGURATION - Single place to control everything
CONFIG = {
    'EXPERIMENT_MODE': 'quick',  # 'quick' or 'full'
    'RANDOM_SEED': 42,

    # Training data
    'n_training_samples': 100000,  # 1000 for quick, 5000 for full

    # Test parameters
    'test_theta_values': [15, 45, 75],  # Will expand for full mode
    'n_experiments_per_theta': 5,       # 5 for quick, 20 for full
    'n_posterior_samples': 10000,

    # Methods to compare
    'methods_to_test': ['raw', 'data_score', 'pairwise_grouped', 'combined_grouped', 'abc', 'abc_adjusted'],

    # Output settings
    'save_results': True,
    'plot_results': True,
    'verbose': True
}

# Auto-expand for full experiment
if CONFIG['EXPERIMENT_MODE'] == 'full':
    CONFIG.update({
        'n_training_samples': 100000,
        'test_theta_values': list(range(5, 100, 10)),
        'n_experiments_per_theta': 20
    })

print(f"Running {CONFIG['EXPERIMENT_MODE']} experiment")
print(f"Testing methods: {CONFIG['methods_to_test']}")

In [ ]:
def estimate_range_by_correlation(x_obs):
    """Estimate range parameter based on spatial correlation of neighboring points in 8x8 grid"""
    locations = create_2d_grid()  # (64, 2) - Grid coordinates

    # Compute empirical correlation proxy for neighboring points
    x_normalized = (x_obs - jnp.mean(x_obs)) / (jnp.std(x_obs) + 1e-8)

    # Use multiple neighbor pairs for correlation estimation - adapted for 8x8 grid
    neighbor_pairs = [
        # Horizontal neighbors (within same row)
        (0,1), (1,2), (2,3), (3,4), (4,5), (5,6), (6,7),  # Row 1
        (8,9), (9,10), (10,11), (11,12), (12,13), (13,14), (14,15),  # Row 2
        (16,17), (24,25), (32,33), (40,41), (48,49), (56,57),  # Sampled from other rows
        # Vertical neighbors (within same column)
        (0,8), (1,9), (2,10), (3,11), (4,12), (5,13), (6,14), (7,15),  # Rows 1-2
        (8,16), (9,17), (16,24), (17,25), (24,32), (32,40), (40,48), (48,56),  # Sampled from other columns
        # Diagonal neighbors
        (0,9), (1,10), (2,11), (8,17), (9,18), (16,25), (24,33)
    ]

    correlations = []
    distances = []

    for i, j in neighbor_pairs:
        if i < 64 and j < 64:  # Ensure indices are within 64-point range
            corr = x_normalized[i] * x_normalized[j]
            dist = jnp.sqrt(jnp.sum((locations[i] - locations[j])**2))  # 2D Euclidean distance
            if corr > 0.01:
                correlations.append(corr)
                distances.append(dist)

    if correlations:
        avg_corr = jnp.mean(jnp.array(correlations))
        avg_dist = jnp.mean(jnp.array(distances))

        # Reverse-estimate range: corr = exp(-dist/range)
        range_est = -avg_dist / jnp.log(avg_corr)

        # Constrain to reasonable range
        return float(jnp.clip(range_est, 5.0, 95.0))

    # If correlations are too small, use prior mean
    return 50.0

In [ ]:
import numpy as np
from numba import jit, prange
from collections import defaultdict

@jit(nopython=True)
def create_2d_grid_numba():
    """Create 8x8 grid points - numba optimized version"""
    coords = np.linspace(0, 1, 8)
    locations = np.zeros((64, 2))
    idx = 0
    for i in range(8):
        for j in range(8):
            locations[idx, 0] = coords[i]
            locations[idx, 1] = coords[j]
            idx += 1
    return locations

@jit(nopython=True)
def estimate_range_by_correlation_numba(x_obs):
    """Estimate range parameter based on 8x8 neighboring points correlation - numba optimized version"""
    locations = create_2d_grid_numba()

    # Calculate empirical correlation proxy for neighboring points
    x_mean = np.mean(x_obs)
    x_std = np.std(x_obs)
    if x_std < 1e-8:
        x_std = 1e-8
    x_normalized = (x_obs - x_mean) / x_std

    # Define neighboring point pairs - use static array instead of list comprehension
    neighbor_pairs = np.array([
        # Horizontal neighbors
        [0,1], [1,2], [2,3], [3,4], [4,5], [5,6], [6,7],
        [8,9], [9,10], [10,11], [11,12], [12,13], [13,14], [14,15],
        [16,17], [17,18], [18,19], [19,20], [20,21], [21,22], [22,23],
        [24,25], [25,26], [26,27], [27,28], [28,29], [29,30], [30,31],
        [32,33], [33,34], [34,35], [35,36], [36,37], [37,38], [38,39],
        [40,41], [41,42], [42,43], [43,44], [44,45], [45,46], [46,47],
        [48,49], [49,50], [50,51], [51,52], [52,53], [53,54], [54,55],
        [56,57], [57,58], [58,59], [59,60], [60,61], [61,62], [62,63],
        # Vertical neighbors
        [0,8], [1,9], [2,10], [3,11], [4,12], [5,13], [6,14], [7,15],
        [8,16], [9,17], [10,18], [11,19], [12,20], [13,21], [14,22], [15,23],
        [16,24], [17,25], [18,26], [19,27], [20,28], [21,29], [22,30], [23,31],
        [24,32], [25,33], [26,34], [27,35], [28,36], [29,37], [30,38], [31,39],
        [32,40], [33,41], [34,42], [35,43], [36,44], [37,45], [38,46], [39,47],
        [40,48], [41,49], [42,50], [43,51], [44,52], [45,53], [46,54], [47,55],
        [48,56], [49,57], [50,58], [51,59], [52,60], [53,61], [54,62], [55,63],
        # Diagonal neighbors (partial)
        [0,9], [1,10], [2,11], [3,12], [4,13], [5,14], [6,15],
        [8,17], [9,18], [10,19], [11,20], [12,21], [13,22], [14,23],
        [16,25], [17,26], [18,27], [19,28], [20,29], [21,30], [22,31]
    ])

    correlations = []
    distances = []

    for k in range(neighbor_pairs.shape[0]):
        i, j = neighbor_pairs[k, 0], neighbor_pairs[k, 1]
        if i < 64 and j < 64:
            corr = x_normalized[i] * x_normalized[j]
            dist_sq = (locations[i, 0] - locations[j, 0])**2 + (locations[i, 1] - locations[j, 1])**2
            dist = np.sqrt(dist_sq)
            if corr > 0.01:
                correlations.append(corr)
                distances.append(dist)

    if len(correlations) > 0:
        avg_corr = np.mean(np.array(correlations))
        avg_dist = np.mean(np.array(distances))

        # Reverse-estimate range: corr = exp(-dist/range)
        if avg_corr > 0:
            range_est = -avg_dist / np.log(avg_corr)
            # Constrain to reasonable range
            return min(max(range_est, 5.0), 95.0)

    # If correlations are too small, use prior mean
    return 50.0

@jit(nopython=True)
def compute_all_distances_and_groups_numba():
    """Calculate all distances and create groups - numba optimized version"""
    locations = create_2d_grid_numba()

    # Pre-compute all distances
    distances = np.zeros((64, 64))
    for i in range(64):
        for j in range(64):
            dist_sq = (locations[i, 0] - locations[j, 0])**2 + (locations[i, 1] - locations[j, 1])**2
            distances[i, j] = np.sqrt(dist_sq)

    # Find unique distances (using simple deduplication logic)
    unique_distances = []
    tolerance = 1e-3

    for i in range(64):
        for j in range(i+1, 64):
            dist = distances[i, j]
            is_unique = True
            for existing_dist in unique_distances:
                if abs(dist - existing_dist) < tolerance:
                    is_unique = False
                    break
            if is_unique:
                unique_distances.append(dist)

    # Sort distances
    unique_distances.sort()

    return distances, np.array(unique_distances)

@jit(nopython=True, parallel=True)
def compute_grouped_scores_for_sample_numba(x, distances, unique_distances, range_param, sigma=5.0):
    """Calculate grouped scores for single sample - numba optimized version"""
    group_scores = np.zeros(len(unique_distances))
    tolerance = 1e-3

    for dist_idx in prange(len(unique_distances)):
        target_dist = unique_distances[dist_idx]
        group_score_sum = 0.0

        # Find all point pairs matching this distance
        for i in range(64):
            for j in range(i+1, 64):
                actual_dist = distances[i, j]
                if abs(actual_dist - target_dist) < tolerance:
                    # Calculate pairwise difference
                    v_ij = x[i] - x[j]

                    # Calculate semivariogram γ(d; θ) = σ²(1 - exp(-d/θ))
                    exp_term = np.exp(-target_dist / range_param)
                    gamma_ij = sigma**2 * (1 - exp_term)
                    gamma_ij = max(gamma_ij, 1e-8)  # Avoid numerical issues

                    # Calculate derivative of γ with respect to range_param
                    dgamma_dtheta = sigma**2 * exp_term * target_dist / (range_param**2)

                    # Calculate pairwise score (based on paper formula 5.8)
                    score_ij = dgamma_dtheta / (2 * gamma_ij) * (v_ij**2 / (2 * gamma_ij) - 1)

                    group_score_sum += score_ij

        group_scores[dist_idx] = group_score_sum

    return group_scores

def compute_distance_grouped_pairwise_scores_estimated_theta_numba(x_samples, tolerance=1e-6):
    """
    Calculate distance-grouped pairwise scores using estimated theta (based on pairwise differences) - numba optimized 64-point version
    """
    print("🔄 Calculating distance-grouped pairwise scores using estimated theta (based on pairwise differences, 64 points, numba optimized)...")

    # Pre-compute distance information
    distances, unique_distances = compute_all_distances_and_groups_numba()

    # Calculate grouped scores for each sample
    all_grouped_scores = []
    sigma = 5.0  # Standard deviation of spatial process

    for sample_idx in range(len(x_samples)):
        x = np.array(x_samples[sample_idx])

        # Estimate theta instead of using true theta
        range_param = estimate_range_by_correlation_numba(x)

        # Calculate grouped scores for this sample
        group_scores = compute_grouped_scores_for_sample_numba(x, distances, unique_distances, range_param, sigma)

        all_grouped_scores.append(group_scores)

    result = np.array(all_grouped_scores)
    print(f"✅ 64-point version grouped pairwise scores calculation completed, output dimensions: {result.shape}")
    print(f"   {len(unique_distances)} dimensions correspond to aggregated scores of point pairs at different 8x8 distances")
    print(f"   Distance groups include: {[round(d, 3) for d in unique_distances]}")

    return result, unique_distances


In [ ]:
# Cell 4: Optimized version - compute pairwise first, then sum to get data score
def prepare_all_data_correlated_simplified_optimized(n_samples, key=42):
    """Prepare correlated data (optimized: compute pairwise first, then sum) - 64-point version"""
    print("Preparing optimized correlated data comparison (64 points)...")
    key = jr.PRNGKey(key)

    # Generate correlated raw data
    theta_train = your_prior_sampler(n_samples, key)
    key, *sim_keys = jr.split(key, n_samples + 1)
    x_train = jnp.array([your_simulator(theta_train[i], sim_keys[i])
                         for i in range(n_samples)])

    # Check data correlation
    print("\nChecking data correlation...")
    corr_matrix = jnp.corrcoef(x_train.T)
    off_diagonal = corr_matrix - jnp.diag(jnp.diag(corr_matrix))
    max_correlation = jnp.max(jnp.abs(off_diagonal))
    print(f"Maximum off-diagonal correlation coefficient: {max_correlation:.4f}")
    if max_correlation > 0.3:
        print("Data confirmed to be correlated")
    else:
        print("Warning: Data correlation is weak")

    print("\nComputing training features (optimized: compute pairwise first, then sum)...")

    # Core optimization: compute pairwise scores grouped by distance first
    print("   Step 1: Compute grouped pairwise features...")
    x_train_pairwise_grouped, distance_groups = compute_distance_grouped_pairwise_scores_estimated_theta_numba(x_train)

    # Then simply sum to get data score (avoid recomputing 2016 pairs!)
    print("   Step 2: Sum pairwise features to get data score...")
    x_train_data_scores = jnp.sum(x_train_pairwise_grouped, axis=1, keepdims=True)  # (n_samples, 1)

    print(f"Computation completed:")
    print(f"   - Pairwise scores for 2016 pairs computed only once")
    print(f"   - Data Score = sum across distance groups")
    print(f"   - Computation time significantly reduced")

    # Step 3. Create combined features - simply concatenate
    print("   Step 3: Create combined features...")

    # Normalize pairwise features
    normalized_pairwise = []
    for i in range(len(x_train_pairwise_grouped)):
        sample_vec = x_train_pairwise_grouped[i]  # vector of distance groups
        normalized_vec = (sample_vec - jnp.mean(sample_vec)) / (jnp.std(sample_vec) + 1e-8)
        normalized_pairwise.append(normalized_vec)
    x_train_pairwise_grouped = jnp.array(normalized_pairwise)

    # Concatenate data_score + normalized_pairwise
    x_train_combined_grouped = jnp.concatenate([x_train_data_scores, x_train_pairwise_grouped], axis=1)

    # Normalize combined features
    normalized_combined = []
    for i in range(len(x_train_combined_grouped)):
        sample_vec = x_train_combined_grouped[i]  # (1 + number of distance groups)-dim vector
        normalized_vec = (sample_vec - jnp.mean(sample_vec)) / (jnp.std(sample_vec) + 1e-8)
        normalized_combined.append(normalized_vec)
    x_train_combined_grouped = jnp.array(normalized_combined)

    print(f"\n64-point optimized training data preparation completed:")
    print(f"   - Parameter shape: {theta_train.shape}")
    print(f"   - Raw data shape: {x_train.shape}")
    print(f"   - Data Score shape: {x_train_data_scores.shape} (obtained via pairwise summation)")
    print(f"   - Pairwise Grouped shape: {x_train_pairwise_grouped.shape}")
    print(f"   - Combined Grouped shape: {x_train_combined_grouped.shape}")
    print(f"   - Number of distance groups: {len(distance_groups)}")

    print(f"\nOptimization effect:")
    print(f"   - Original version: required 2016*2 = 4032 pairwise score computations")
    print(f"   - Optimized version: only 2016*1 = 2016 computations")
    print(f"   - Speed improvement: ~2x")

    return (theta_train, x_train, x_train_data_scores,
            x_train_pairwise_grouped, x_train_combined_grouped, distance_groups)




In [ ]:
def generate_training_data():
    """Generate training data using existing functions"""
    print(f"Generating {CONFIG['n_training_samples']} training samples...")

    key = jr.PRNGKey(CONFIG['RANDOM_SEED'])
    theta_train = your_prior_sampler(CONFIG['n_training_samples'], key)

    key, *sim_keys = jr.split(key, CONFIG['n_training_samples'] + 1)
    x_train = jnp.array([your_simulator(theta_train[i], sim_keys[i])
                        for i in range(CONFIG['n_training_samples'])])

    print(f"Training data shape: theta={theta_train.shape}, x={x_train.shape}")
    return theta_train, x_train

# Generate once, use for all methods
theta_train, x_train = generate_training_data()

In [ ]:
def prepare_all_features(theta_train, x_train):
    """Prepare all feature types using existing optimized functions"""
    print("Preparing features for all methods...")

    # Use your existing optimized feature computation
    (theta_train, x_train, x_train_data_scores,
     x_train_pairwise_grouped, x_train_combined_grouped,
     distance_groups) = prepare_all_data_correlated_simplified_optimized(
        n_samples=CONFIG['n_training_samples'],
        key=CONFIG['RANDOM_SEED']
    )

    features = {
        'raw': x_train,
        'data_score': x_train_data_scores,
        'pairwise_grouped': x_train_pairwise_grouped,
        'combined_grouped': x_train_combined_grouped
    }

    print("Feature dimensions:")
    for name, feat in features.items():
        print(f"  {name}: {feat.shape}")

    return features

features = prepare_all_features(theta_train, x_train)

In [ ]:
def train_single_method(method_name, theta_train, x_input, train_key, dim_data=64):  # 🔥 changed from 16 to 64
    """Train a single method"""
    print(f"\n Training {method_name} method...")
    start_time = time.time()

    key = jr.PRNGKey(train_key)
    config = create_range_parameter_config()

    # Adjust data dimension
    config.algorithm.dim_data = dim_data

    sde = get_sde(config)
    model = NCMLP(key, config)

    trained_model, ds_mean, ds_std = train_score_network(
        config, model, sde, theta_train, x_input, key
    )

    training_time = time.time() - start_time
    print(f"{method_name} training completed, time taken: {training_time:.2f} seconds")

    return {
        'trained_model': trained_model,
        'ds_mean': ds_mean,
        'ds_std': ds_std,
        'sde': sde,
        'config': config,
        'training_time': training_time,
        'method': method_name
    }




In [ ]:
def train_all_cnf_methods(theta_train, features):
    """Train CNF models for all feature types"""
    print("Training CNF models...")

    trained_models = {}

    for method_name in ['raw', 'data_score', 'pairwise_grouped', 'combined_grouped']:
        if method_name not in CONFIG['methods_to_test']:
            continue

        print(f"\nTraining {method_name} model...")
        start_time = time.time()

        # Use your existing training function
        x_input = features[method_name]
        dim_data = x_input.shape[1]

        trained_models[method_name] = train_single_method(
            method_name, theta_train, x_input,
            train_key=CONFIG['RANDOM_SEED'],
            dim_data=dim_data
        )

        training_time = time.time() - start_time
        print(f"Training completed in {training_time:.1f}s")

    return trained_models

trained_models = train_all_cnf_methods(theta_train, features)

In [ ]:
def create_cnf_samplers(trained_models):
    """Create CNF samplers from trained models"""
    print("Creating CNF samplers...")

    cnf_samplers = {}

    for method_name, model_info in trained_models.items():
        cnf = CNF(
            score_network=model_info['trained_model'],
            sde=model_info['sde'],
            ds_means=model_info['ds_mean'],
            ds_stds=model_info['ds_std']
        )

        cnf_samplers[method_name] = {
            'cnf': cnf,
            'config': model_info['config'],
            'training_time': model_info['training_time']
        }

    print(f"Created {len(cnf_samplers)} CNF samplers")
    return cnf_samplers

cnf_samplers = create_cnf_samplers(trained_models)

In [ ]:
def run_single_inference(method_name, x_obs, cnf_samplers=None):
    """Run inference for a single method"""

    if method_name in ['abc', 'abc_adjusted']:
        # Use your existing ABC functions
        use_adjustment = (method_name == 'abc_adjusted')
        samples = run_abc_inference_with_adjustment(
            x_obs,
            n_samples=CONFIG['n_posterior_samples'],
            apply_adjustment=use_adjustment,
            verbose=False
        )
        return samples, 0.0  # ABC has no training time

    else:
        # CNF methods
        cnf_info = cnf_samplers[method_name]

        # Prepare features using your existing function
        x_input = compute_test_input_fixed_optimized_numba(x_obs, method_name)

        # Sample from posterior
        key = jr.PRNGKey(np.random.randint(0, 1000000))
        samples = cnf_info['cnf'].batch_sample_fn(
            CONFIG['n_posterior_samples'], x_input, key
        )

        return samples, cnf_info['training_time']

In [ ]:
import numpy as np
import time
from numba import jit, prange
from collections import defaultdict

@jit(nopython=True)
def create_2d_grid_numba():
    """创建8x8网格点 - numba优化版本"""
    coords = np.linspace(0, 1, 8)
    locations = np.zeros((64, 2))
    idx = 0
    for i in range(8):
        for j in range(8):
            locations[idx, 0] = coords[i]
            locations[idx, 1] = coords[j]
            idx += 1
    return locations

@jit(nopython=True)
def estimate_range_by_correlation_numba(x_obs):
    """基于8x8相邻点相关性估计range parameter - numba优化版本"""
    locations = create_2d_grid_numba()

    # 计算相邻点的经验相关性代理
    x_mean = np.mean(x_obs)
    x_std = np.std(x_obs)
    if x_std < 1e-8:
        x_std = 1e-8
    x_normalized = (x_obs - x_mean) / x_std

    # 定义相邻点对 - 使用静态数组
    neighbor_pairs = np.array([
        # 水平相邻 (部分)
        [0,1], [1,2], [2,3], [8,9], [9,10], [16,17], [24,25],
        # 垂直相邻 (部分)
        [0,8], [1,9], [8,16], [16,24], [24,32], [32,40], [40,48], [48,56],
        # 对角相邻 (部分)
        [0,9], [1,10], [8,17], [16,25]
    ])

    correlations = []
    distances = []

    for k in range(neighbor_pairs.shape[0]):
        i, j = neighbor_pairs[k, 0], neighbor_pairs[k, 1]
        if i < 64 and j < 64:
            corr = x_normalized[i] * x_normalized[j]
            dist_sq = (locations[i, 0] - locations[j, 0])**2 + (locations[i, 1] - locations[j, 1])**2
            dist = np.sqrt(dist_sq)
            if corr > 0.01:
                correlations.append(corr)
                distances.append(dist)

    if len(correlations) > 0:
        avg_corr = np.mean(np.array(correlations))
        avg_dist = np.mean(np.array(distances))

        # 反推range: corr = exp(-dist/range)
        if avg_corr > 0:
            range_est = -avg_dist / np.log(avg_corr)
            # 限制在合理范围内
            return min(max(range_est, 5.0), 95.0)

    # 如果相关性都太小，使用先验均值
    return 50.0

@jit(nopython=True)
def compute_distance_groups_numba():
    """预计算所有距离分组信息 - numba优化版本"""
    locations = create_2d_grid_numba()

    # 计算所有距离并找到唯一值
    unique_distances = []
    tolerance = 1e-3

    for i in range(64):
        for j in range(i+1, 64):
            dist_sq = (locations[i, 0] - locations[j, 0])**2 + (locations[i, 1] - locations[j, 1])**2
            dist = np.sqrt(dist_sq)

            # 检查是否为新的唯一距离
            is_unique = True
            for existing_dist in unique_distances:
                if abs(dist - existing_dist) < tolerance:
                    is_unique = False
                    break
            if is_unique:
                unique_distances.append(dist)

    # 排序距离
    unique_distances.sort()
    return np.array(unique_distances), locations

@jit(nopython=True, parallel=True)
def compute_pairwise_grouped_scores_numba(x_obs, theta_estimated, unique_distances, locations):
    """计算距离分组的pairwise scores - numba优化版本"""
    group_scores = np.zeros(len(unique_distances))
    sigma = 5.0
    tolerance = 1e-3

    # 并行计算每个距离组的分数
    for dist_idx in prange(len(unique_distances)):
        target_dist = unique_distances[dist_idx]
        group_score_sum = 0.0

        # 找到所有匹配这个距离的点对
        for i in range(64):
            for j in range(i+1, 64):
                actual_dist_sq = (locations[i, 0] - locations[j, 0])**2 + (locations[i, 1] - locations[j, 1])**2
                actual_dist = np.sqrt(actual_dist_sq)

                if abs(actual_dist - target_dist) < tolerance:
                    # 计算pairwise score
                    v_ij = x_obs[i] - x_obs[j]
                    exp_term = np.exp(-target_dist / theta_estimated)
                    gamma_ij = sigma**2 * (1 - exp_term)
                    gamma_ij = max(gamma_ij, 1e-8)
                    dgamma_dtheta = sigma**2 * exp_term * target_dist / (theta_estimated**2)
                    score_ij = dgamma_dtheta / (2 * gamma_ij) * (v_ij**2 / (2 * gamma_ij) - 1)

                    group_score_sum += score_ij

        group_scores[dist_idx] = group_score_sum

    return group_scores

@jit(nopython=True)
def normalize_array_numba(arr):
    """数组标准化 - numba优化版本"""
    mean_val = np.mean(arr)
    std_val = np.std(arr)
    if std_val < 1e-8:
        std_val = 1e-8
    return (arr - mean_val) / std_val

# 预计算静态数据（只需要计算一次）
_unique_distances, _locations = compute_distance_groups_numba()

def compute_test_input_fixed_optimized_numba(x_obs, method):
    """优化版本的测试输入计算函数 - numba优化版本"""
    x_obs_array = np.array(x_obs)

    if method == 'raw':
        return x_obs_array  # 64维原始数据

    # 估计theta (8x8版本)
    theta_estimated = estimate_range_by_correlation_numba(x_obs_array)

    # 只计算一次pairwise grouped scores (避免重复计算2016个点对!)
    pairwise_scores_raw = compute_pairwise_grouped_scores_numba(
        x_obs_array, theta_estimated, _unique_distances, _locations
    )

    if method == 'data_score':
        # 简单求和得到data score
        total_score = np.sum(pairwise_scores_raw)
        return np.array([total_score])

    elif method == 'pairwise_grouped':
        # 直接返回归一化的pairwise scores
        normalized_result = normalize_array_numba(pairwise_scores_raw)
        return normalized_result

    elif method == 'combined_grouped':
        # 组合：data_score + normalized_pairwise
        data_score_part = np.array([np.sum(pairwise_scores_raw)])  # 1维

        pairwise_part = normalize_array_numba(pairwise_scores_raw)

        # 合并并归一化
        combined_result = np.concatenate((data_score_part, pairwise_part))
        combined_result = normalize_array_numba(combined_result)
        return combined_result

    else:
        raise ValueError(f"Unknown method: {method}")

In [ ]:
def compute_test_input_fixed_optimized_numba(x_obs, method):
    """优化版本的测试输入计算函数 - numba优化版本"""
    x_obs_array = np.array(x_obs)

    if method == 'raw':
        return x_obs_array  # 64维原始数据

    # 估计theta (8x8版本)
    theta_estimated = estimate_range_by_correlation_numba(x_obs_array)

    # 只计算一次pairwise grouped scores (避免重复计算2016个点对!)
    pairwise_scores_raw = compute_pairwise_grouped_scores_numba(
        x_obs_array, theta_estimated, _unique_distances, _locations
    )

    if method == 'data_score':
        # 简单求和得到data score
        total_score = np.sum(pairwise_scores_raw)
        return np.array([total_score])

    elif method == 'pairwise_grouped':
        # 直接返回归一化的pairwise scores
        normalized_result = normalize_array_numba(pairwise_scores_raw)
        return normalized_result

    elif method == 'combined_grouped':
        # 组合：data_score + normalized_pairwise
        data_score_part = np.array([np.sum(pairwise_scores_raw)])  # 1维

        pairwise_part = normalize_array_numba(pairwise_scores_raw)

        # 合并并归一化
        combined_result = np.concatenate((data_score_part, pairwise_part))
        combined_result = normalize_array_numba(combined_result)
        return combined_result

    else:
        raise ValueError(f"Unknown method: {method}")



In [ ]:
# 在原有的4种方法基础上添加ABC作为第5种方法
import numpy as np
import pyabc
from pyabc import ABCSMC, DistributionBase, Parameter
from scipy.stats import uniform
import tempfile
import os

# 首先运行ABC
print("🔬 运行ABC推断...")

# 使用你的完整ABC实现
def gaussian_field_model(params):
    """Gaussian Field模拟器 - 返回原始数据"""
    # 添加调试计数器
    if not hasattr(gaussian_field_model, '_call_count'):
        gaussian_field_model._call_count = 0
        gaussian_field_model._success_count = 0
    gaussian_field_model._call_count += 1

    # 转换参数
    theta = np.array([params['range_param']])

    # 验证参数
    if theta[0] <= 0 or theta[0] >= 100:
        if gaussian_field_model._call_count <= 5:
            print(f"模拟器 #{gaussian_field_model._call_count}: 参数验证失败")
        # 返回简单的替代数据而不是NaN
        return {"data": np.full(64, 0.0, dtype=np.float64)}

    try:
        # 生成模拟数据
        key = jr.PRNGKey(np.random.randint(0, 1000000))
        sim_data = your_simulator(jnp.array(theta), key)  # (64,)

        if sim_data is None or np.any(np.isnan(sim_data)) or np.any(np.isinf(sim_data)):
            if gaussian_field_model._call_count <= 5:
                print(f"模拟器 #{gaussian_field_model._call_count}: 模拟数据无效")
            # 返回简单的替代数据而不是NaN
            return {"data": np.full(64, 0.0, dtype=np.float64)}

        # 成功生成数据
        gaussian_field_model._success_count += 1
        if gaussian_field_model._call_count <= 5:
            print(f"模拟器 #{gaussian_field_model._call_count}: 成功生成数据，形状={sim_data.shape}")

        # 确保返回正确的数据类型，并进行数值检查
        sim_array = np.array(sim_data, dtype=np.float64)

        # 替换任何剩余的NaN或inf值
        if np.any(np.isnan(sim_array)) or np.any(np.isinf(sim_array)):
            print(f"发现无效值，使用替代数据")
            return {"data": np.full(64, 0.0, dtype=np.float64)}

        return {"data": sim_array}

    except Exception as e:
        if gaussian_field_model._call_count <= 5:
            print(f"模拟器 #{gaussian_field_model._call_count}: 异常 - {str(e)}")
        # 返回简单的替代数据而不是NaN
        return {"data": np.full(64, 0.0, dtype=np.float64)}


class GaussianFieldPrior(DistributionBase):
    """Gaussian Field先验分布"""

    def __init__(self):
        # 简单的uniform先验 range ~ uniform(0, 100)
        self.range_min = 0.0
        self.range_max = 100.0
        self.uniform_dist = uniform(loc=self.range_min, scale=self.range_max - self.range_min)

        self.param_names = ['range_param']

    def rvs(self, *args, **kwargs):
        """采样 - 简单的均匀分布"""
        max_attempts = 100
        for attempt in range(max_attempts):
            # 从均匀分布采样
            raw_sample = self.uniform_dist.rvs()

            # 验证参数
            if self._validate_sample(raw_sample):
                sample_dict = {'range_param': float(raw_sample)}
                return Parameter(sample_dict)

        # 如果所有尝试都失败，返回中间值
        print(f"先验采样失败，使用中间值")
        sample_dict = {'range_param': 50.0}
        return Parameter(sample_dict)

    def pdf(self, x):
        """计算密度 - 均匀分布"""
        if not isinstance(x, dict):
            x = x.to_dict() if hasattr(x, 'to_dict') else dict(x)

        # 转换为数组
        theta = x['range_param']

        # 验证参数约束
        if not self._validate_sample(theta):
            return 1e-10

        # 均匀分布密度
        try:
            density = self.uniform_dist.pdf(theta)
            density_val = max(float(density), 1e-10)

            # 检查密度是否有效
            if np.isnan(density_val) or np.isinf(density_val):
                return 1e-10

            return density_val
        except Exception:
            return 1e-10

    def _validate_sample(self, theta):
        """参数验证"""
        # range参数范围
        if theta <= 0.0 or theta >= 100.0:
            return False

        # 检查数值稳定性
        if np.isnan(theta) or np.isinf(theta):
            return False

        return True


def gaussian_field_distance(x, y):
    """距离函数 - 使用combined_grouped作为汇总统计"""
    data_x = x["data"]  # (64,)
    data_y = y["data"]  # (64,)

    # 检查数据有效性
    if np.any(np.isnan(data_x)) or np.any(np.isnan(data_y)):
        return 1e8

    if np.any(np.isinf(data_x)) or np.any(np.isinf(data_y)):
        return 1e8

    try:
        # 使用combined_grouped作为summary statistics
        # 确保数组类型正确
        data_x = np.asarray(data_x, dtype=np.float64)
        data_y = np.asarray(data_y, dtype=np.float64)
        summary_x = compute_test_input_fixed_optimized_numba(data_x, 'combined_grouped')
        summary_y = compute_test_input_fixed_optimized_numba(data_y, 'combined_grouped')

        # 检查汇总统计是否有效
        if np.any(np.isnan(summary_x)) or np.any(np.isnan(summary_y)):
            return 1e8

        if np.any(np.isinf(summary_x)) or np.any(np.isinf(summary_y)):
            return 1e8

        # 欧式距离比较汇总统计
        distance = np.sqrt(np.sum((summary_x - summary_y)**2))

        # 检查距离是否有效
        if np.isnan(distance) or np.isinf(distance) or distance < 0:
            return 1e8

        return float(distance)

    except Exception as e:
        return 1e10  # 明确表示这个样本不好


In [ ]:
# Complete ABC Regression Adjustment Integration Code
import numpy as np
import pyabc
from pyabc import ABCSMC, DistributionBase, Parameter
from scipy.stats import uniform, gaussian_kde
import tempfile
import os
import matplotlib.pyplot as plt
import time
import jax.numpy as jnp
import jax.random as jr
from collections import defaultdict

# ========== Helper Functions ==========
def _coerce_to_array(item, target_shape):
    """Convert various formats to numpy array"""
    try:
        if hasattr(item, 'numpy'):  # JAX/PyTorch
            arr = np.array(item.numpy(), dtype=float)
        elif hasattr(item, 'values'):  # pandas
            arr = np.array(item.values, dtype=float)
        else:
            arr = np.array(item, dtype=float)

        # Try to reshape to target shape
        if arr.shape != target_shape:
            if arr.size == np.prod(target_shape):
                arr = arr.reshape(target_shape)
            else:
                return None
        return arr
    except:
        return None

def _epanechnikov_weights(z, bandwidth):
    """Epanechnikov kernel weights"""
    abs_z = np.abs(z / bandwidth)
    w = np.where(abs_z <= 1, 0.75 * (1 - abs_z**2), 0)
    return w

def _ess(weights):
    """Effective sample size"""
    w = np.asarray(weights)
    w = w[np.isfinite(w) & (w > 0)]
    if len(w) == 0:
        return 0
    return np.sum(w)**2 / np.sum(w**2)

def _wls(X, y, weights, ridge=1e-8):
    """Weighted least squares"""
    W = np.diag(weights)
    XtWX = X.T @ W @ X
    XtWX += ridge * np.eye(XtWX.shape[0])
    XtWy = X.T @ W @ y

    try:
        beta = np.linalg.solve(XtWX, XtWy)
        yhat = X @ beta

        # R-squared calculation
        y_mean = np.average(y, weights=weights)
        ss_tot = np.sum(weights * (y - y_mean)**2)
        ss_res = np.sum(weights * (y - yhat)**2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0

        return beta, yhat, r2
    except:
        return np.zeros(X.shape[1]), np.zeros(len(y)), 0

# ========== Core Regression Adjustment Function ==========
def abc_regression_adjustment(
    theta_samples,
    summary_stats_list,
    observed_summary_matrix,
    bandwidth_quantile=0.8,
    min_effective_n=30,
    ridge=0,
    feature_clip=6.0,
    verbose=False
):
    """
    ABC regression adjustment - adapted for your code structure
    """
    theta_samples = np.asarray(theta_samples, dtype=float)
    n_samples, n_params = theta_samples.shape

    obs = np.asarray(observed_summary_matrix, dtype=float)
    obs_shape = obs.shape
    obs_flat = obs.reshape(-1)
    obs_finite_flat = np.isfinite(obs_flat)

    if verbose:
        print(f"ABC regression adjustment: {n_samples} samples, feature dimension {obs_shape}")

    # ========== Calculate distances ==========
    dists = np.full(n_samples, np.inf, dtype=float)
    coerced_list = [None] * n_samples
    converted, failed = 0, 0

    for k in range(n_samples):
        if k >= len(summary_stats_list):
            failed += 1
            continue

        sim = _coerce_to_array(summary_stats_list[k], obs_shape)
        if sim is None:
            failed += 1
            continue

        coerced_list[k] = sim
        sim_flat = sim.reshape(-1)
        both = obs_finite_flat & np.isfinite(sim_flat)
        if not np.any(both):
            failed += 1
            continue

        diff = (sim_flat - obs_flat)[both]
        dists[k] = np.sqrt(np.sum(diff * diff))
        converted += 1

    if verbose:
        print(f"Distance calculation: succeeded={converted}, failed={failed}")

    valid = np.isfinite(dists)
    if not np.any(valid):
        if verbose:
            print("Error: no valid distance calculations")
        return theta_samples.copy()

    # ========== Standardize distances and bandwidth ==========
    z = dists[valid] - dists[valid].mean()
    std = dists[valid].std()
    z = z / std if std > 0 else np.zeros_like(z)

    bandwidth = np.quantile(np.abs(z), bandwidth_quantile)
    if not np.isfinite(bandwidth) or bandwidth <= 0:
        bandwidth = 1.0

    z_all = np.full_like(dists, np.nan, dtype=float)
    z_all[valid] = (dists[valid] - dists[valid].mean()) / (std if std > 0 else 1.0)

    # ========== Kernel weights ==========
    w = _epanechnikov_weights(z_all, bandwidth)
    keep_idx = np.where(w > 0)[0]
    ess = _ess(w)

    if verbose:
        print(f"Bandwidth: {bandwidth:.6f}, effective samples: {keep_idx.size}, ESS: {ess:.1f}")

    if keep_idx.size < 5:
        if verbose:
            print("Too few effective samples, returning original samples")
        return theta_samples.copy()

    # ========== Construct design matrix ==========
    mask = np.isfinite(obs_flat)
    p = int(mask.sum())

    X_rows = []
    kept_ok = []
    for k in keep_idx:
        sim = coerced_list[k]
        if sim is None:
            continue
        sim2_flat = sim.reshape(-1).copy()
        bad = ~np.isfinite(sim2_flat)
        if np.any(bad):
            sim2_flat[bad] = obs_flat[bad]
        diff_vec = (sim2_flat - obs_flat)[mask]
        X_rows.append(diff_vec)
        kept_ok.append(k)

    if len(X_rows) < 5:
        if verbose:
            print("Insufficient samples after design matrix construction")
        return theta_samples.copy()

    kept_ok = np.asarray(kept_ok, dtype=int)
    X = np.asarray(X_rows)

    # Column standardization
    col_mean = X.mean(axis=0)
    col_std = X.std(axis=0, ddof=0)
    col_std[col_std == 0] = 1.0
    Xs = (X - col_mean) / col_std

    if feature_clip is not None and feature_clip > 0:
        np.clip(Xs, -feature_clip, feature_clip, out=Xs)

    # Add intercept
    Xs = np.column_stack([np.ones(Xs.shape[0]), Xs])

    # ========== Parameter-wise regression adjustment ==========
    theta_adj = theta_samples.copy()

    for pidx in range(n_params):
        if verbose:
            print(f"Adjusting parameter {pidx+1}/{n_params}")

        y = theta_samples[kept_ok, pidx]
        beta, yhat, r2 = _wls(Xs, y, w[kept_ok], ridge=ridge)

        if verbose:
            print(f"  R²: {r2:.4f}")

        # Only adjust samples with kernel weights
        for k in kept_ok:
            sim = coerced_list[k]
            if sim is None:
                continue
            sim2_flat = sim.reshape(-1).copy()
            bad = ~np.isfinite(sim2_flat)
            if np.any(bad):
                sim2_flat[bad] = obs_flat[bad]
            diff_vec = (sim2_flat - obs_flat)[mask]
            xk = (diff_vec - col_mean) / col_std
            if feature_clip is not None and feature_clip > 0:
                np.clip(xk, -feature_clip, feature_clip, out=xk)
            xk = np.concatenate([[1.0], xk])
            yk_pred = xk @ beta
            adjustment = yk_pred - beta[0]
            theta_adj[k, pidx] = theta_samples[k, pidx] - adjustment

    if verbose:
        print("Regression adjustment complete!")

    return theta_adj

# ========== Enhanced ABC Implementation ==========
def run_abc_inference_with_adjustment(x_obs, n_samples, apply_adjustment=True, verbose=False):
    """
    Enhanced ABC inference with regression adjustment
    """
    if verbose:
        print(f"Starting ABC inference (samples: {n_samples}, regression adjustment: {apply_adjustment})")

    # Reset simulator counters
    if hasattr(gaussian_field_model, '_call_count'):
        gaussian_field_model._call_count = 0
        gaussian_field_model._success_count = 0

    # ABC setup
    prior = GaussianFieldPrior()

    abc = ABCSMC(
        models=gaussian_field_model,
        parameter_priors=prior,
        distance_function=gaussian_field_distance,
        population_size= n_samples,
        sampler=pyabc.sampler.SingleCoreSampler(),
        eps=pyabc.epsilon.QuantileEpsilon(alpha=0.5, quantile_multiplier=0.95)
    )

    # Prepare observed data
    obs = {"data": np.array(x_obs, dtype=np.float64)}

    # Create database
    db_path = os.path.join(tempfile.gettempdir(), f"abc_adjusted_{int(time.time())}.db")
    abc.new("sqlite:///" + db_path, obs)

    if verbose:
        print("Running ABC sampling...")

    try:
        # Run ABC
        history = abc.run(max_nr_populations=3)

        # Extract samples
        df, w = history.get_distribution(m=0, t=history.max_t)
        theta_samples = df['range_param'].values

        if verbose:
            print(f"ABC complete: {len(theta_samples)} samples")

        # ========== Apply regression adjustment ==========
        if apply_adjustment and len(theta_samples) > 50:
            if verbose:
                print("Starting regression adjustment...")

            # Recompute summary statistics for regression adjustment
            summary_stats_list = []
            obs_summary = compute_test_input_fixed_optimized_numba(x_obs, 'combined_grouped')

            # Recompute summary statistics for each ABC sample
            n_for_adjustment = min(len(theta_samples), 500)  # Limit number to avoid slowdown
            for i in range(n_for_adjustment):
                theta_val = theta_samples[i]
                if verbose and i % 100 == 0:
                    print(f"  Recomputing summary statistics {i+1}/{n_for_adjustment}")

                try:
                    # Simulate data with this theta value
                    key = jr.PRNGKey(np.random.randint(0, 1000000))
                    sim_data = your_simulator(jnp.array([theta_val]), key)
                    summary_stats = compute_test_input_fixed_optimized_numba(sim_data, 'combined_grouped')
                    summary_stats_list.append(summary_stats)
                except Exception as e:
                    if verbose and i < 5:
                        print(f"    Sample {i} simulation failed: {e}")
                    # Use observed values when failed
                    summary_stats_list.append(obs_summary.copy())

            # Only adjust samples for which we recomputed summary statistics
            theta_for_adjustment = theta_samples[:n_for_adjustment].reshape(-1, 1)

            # Apply regression adjustment
            theta_adjusted = abc_regression_adjustment(
                theta_samples=theta_for_adjustment,
                summary_stats_list=summary_stats_list,
                observed_summary_matrix=obs_summary,
                verbose=verbose
            )

            # Combine adjusted and unadjusted samples
            final_samples = np.concatenate([
                theta_adjusted.flatten(),
                theta_samples[n_for_adjustment:]
            ])

            if verbose:
                orig_mean = np.mean(theta_samples)
                adj_mean = np.mean(final_samples)
                print(f"Adjustment effect: original mean={orig_mean:.3f}, adjusted mean={adj_mean:.3f}")
        else:
            final_samples = theta_samples
            if verbose and apply_adjustment:
                print("Insufficient samples, skipping regression adjustment")

        # Randomly sample specified number if needed
        if len(final_samples) > n_samples:
            indices = np.random.choice(len(final_samples), n_samples, replace=False)
            final_samples = final_samples[indices]

        if verbose:
            print(f"Returning {len(final_samples)} samples")

        # Clean up temporary database
        try:
            os.remove(db_path)
        except:
            pass

        return jnp.array(final_samples)

    except Exception as e:
        if verbose:
            print(f"ABC failed: {str(e)}")
        # Clean up temporary database
        try:
            os.remove(db_path)
        except:
            pass
        return jr.uniform(jr.PRNGKey(42), (n_samples,), minval=5.0, maxval=95.0)

# ========== Update original ABC function for compatibility ==========
def run_abc_inference(x_obs, n_samples=1000, use_adjustment=False):
    """
    Updated ABC inference function - compatible with MSE experiments
    """
    return run_abc_inference_with_adjustment(
        x_obs, n_samples=n_samples,
        apply_adjustment=use_adjustment,
        verbose=False
    )



In [ ]:
def run_single_inference(method_name, x_obs, cnf_samplers=None):
    """Run inference for a single method"""

    start_time = time.time()  # 开始计时

    if method_name in ['abc', 'abc_adjusted']:
        use_adjustment = (method_name == 'abc_adjusted')
        samples = run_abc_inference_with_adjustment(
            x_obs,
            n_samples=CONFIG['n_posterior_samples'],
            apply_adjustment=use_adjustment,
            verbose=False
        )
        inference_time = time.time() - start_time  # 计算推理时间
        return samples, inference_time, 0.0, CONFIG['n_posterior_samples']
        # 返回：样本，推理时间，训练时间(0)，训练样本数(就是后验样本数)

    else:
        cnf_info = cnf_samplers[method_name]
        x_input = compute_test_input_fixed_optimized_numba(x_obs, method_name)

        key = jr.PRNGKey(np.random.randint(0, 1000000))
        samples = cnf_info['cnf'].batch_sample_fn(
            CONFIG['n_posterior_samples'], x_input, key
        )

        inference_time = time.time() - start_time  # 计算推理时间
        return samples, inference_time, cnf_info['training_time'], CONFIG['n_training_samples']
        # 返回：样本，推理时间，训练时间，训练样本数

In [ ]:
def run_full_experiment():
    """Run the complete comparison experiment"""
    print("Starting full comparison experiment...")
    print(f"Testing theta values: {CONFIG['test_theta_values']}")
    print(f"Methods: {CONFIG['methods_to_test']}")

    results = {method: {
    'estimates': [],
    'inference_times': [],    # 改名：只存推理时间
    'training_time': 0,       # 总训练时间
    'n_training_samples': 0   # 新增：训练样本数
} for method in CONFIG['methods_to_test']}

    total_experiments = len(CONFIG['test_theta_values']) * CONFIG['n_experiments_per_theta']
    experiment_count = 0

    for theta_idx, true_theta in enumerate(CONFIG['test_theta_values']):
        print(f"\nTesting theta = {true_theta} ({theta_idx+1}/{len(CONFIG['test_theta_values'])})")

        for exp_idx in range(CONFIG['n_experiments_per_theta']):
            experiment_count += 1

            if CONFIG['verbose'] and exp_idx % 5 == 0:
                print(f"  Experiment {exp_idx+1}/{CONFIG['n_experiments_per_theta']}")

            # Generate test observation
            key = jr.PRNGKey(CONFIG['RANDOM_SEED'] + experiment_count)
            theta_array = jnp.array([float(true_theta)])
            x_obs = your_simulator(theta_array, key)

            # Test each method
            for method_name in CONFIG['methods_to_test']:
                try:
                    # 接收4个返回值
                    samples, inference_time, training_time, n_training_samples = run_single_inference(
                        method_name, x_obs, cnf_samplers
                    )

                    # 存储结果
                    results[method_name]['inference_times'].append(inference_time)
                    results[method_name]['training_time'] = training_time
                    results[method_name]['n_training_samples'] = n_training_samples

                    # Store results
                    posterior_mean = float(jnp.mean(samples))
                    results[method_name]['estimates'].append({
                        'true_theta': true_theta,
                        'estimate': posterior_mean,
                        'samples': samples
                    })


                except Exception as e:
                    print(f"    {method_name} failed: {str(e)}")

    return results

# Run the experiment
experiment_results = run_full_experiment()

In [ ]:
def analyze_results(results):
    """Analyze and display results"""
    print("\n" + "="*80)
    print("EXPERIMENT RESULTS SUMMARY")
    print("="*80)

    # Calculate MSE and timing for each method
    mse_results = {}
    timing_results = {}

    for method_name, method_results in results.items():
        estimates_data = method_results['estimates']

        # 计算训练成本分摊
        training_time = method_results['training_time']
        n_training_samples = method_results['n_training_samples']
        if n_training_samples > 0:
            training_cost_per_sample = training_time / n_training_samples
        else:
            training_cost_per_sample = 0

        # Group by theta value
        theta_groups = {}
        theta_times = {}

        for i, est in enumerate(estimates_data):
            theta = est['true_theta']
            if theta not in theta_groups:
                theta_groups[theta] = []
                theta_times[theta] = []
            theta_groups[theta].append(est['estimate'])
            theta_times[theta].append(method_results['inference_times'][i])

        # Calculate MSE and costs for each theta
        method_mse = {}
        method_total_costs = {}
        for theta in theta_groups.keys():
            estimates = theta_groups[theta]
            mse = np.mean([(est - theta)**2 for est in estimates])
            method_mse[theta] = mse

            inference_time = np.mean(theta_times[theta])
            total_cost = training_cost_per_sample + inference_time
            method_total_costs[theta] = total_cost

        mse_results[method_name] = method_mse
        timing_results[method_name] = method_total_costs

        # Print results for each theta
        print(f"\n{method_name.upper()}:")
        for theta in sorted(method_mse.keys()):
            inference_time = np.mean(theta_times[theta])
            print(f"  θ={theta}: MSE={method_mse[theta]:.4f}")
            print(f"           Training Cost={training_cost_per_sample:.6f}s, Inference Cost={inference_time:.6f}s, Total Cost={method_total_costs[theta]:.6f}s")

    # 计算相对效率
    print("\n" + "="*80)
    print("RELATIVE EFFICIENCY ANALYSIS")
    print("="*80)

    # 找到基准方法 (通常选择一个作为参考)
    baseline_method = list(mse_results.keys())[0]  # 使用第一个方法作为基准

    for theta in sorted(mse_results[baseline_method].keys()):
        print(f"\nθ={theta}:")
        baseline_mse = mse_results[baseline_method][theta]
        baseline_cost = timing_results[baseline_method][theta]

        for method_name in mse_results.keys():
            method_mse = mse_results[method_name][theta]
            method_cost = timing_results[method_name][theta]

            # 相对效率 = (baseline_mse / method_mse) / (method_cost / baseline_cost)
            # = (baseline_mse / method_mse) * (baseline_cost / method_cost)
            relative_efficiency = (baseline_mse / method_mse) * (baseline_cost / method_cost)

            print(f"  {method_name}: Relative Efficiency = {relative_efficiency:.3f}")

    return mse_results, timing_results

# 修改绘图函数
def plot_comparison_results(mse_results, timing_results):
    """Plot comparison results"""
    if not CONFIG['plot_results']:
        return

    plt.figure(figsize=(12, 5))

    # MSE plot
    plt.subplot(1, 2, 1)
    for method_name, method_mse in mse_results.items():
        thetas = sorted(method_mse.keys())
        mses = [method_mse[theta] for theta in thetas]
        plt.plot(thetas, mses, 'o-', label=method_name, linewidth=2)

    plt.xlabel('True θ')
    plt.ylabel('MSE')
    plt.title('Mean Squared Error vs True θ')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')

    # Cost plot - 显示平均成本
    plt.subplot(1, 2, 2)
    methods = list(timing_results.keys())
    # 计算每个方法的平均成本
    avg_costs = [np.mean(list(timing_results[method].values())) for method in methods]

    plt.bar(methods, avg_costs)
    plt.xlabel('Method')
    plt.ylabel('Average Total Cost per Sample (s)')
    plt.title('Average Computational Cost Comparison')
    plt.xticks(rotation=45)
    plt.yscale('log')

    plt.tight_layout()
    plt.show()
# 调用
mse_results, cost_results = analyze_results(experiment_results)
plot_comparison_results(mse_results, cost_results)
